# 15 Exception handling
see Section 14.4 of the textbook

There are 2 main reasons a program may fail to run:
   * *syntax error*: the code is not valid python.
   * *Exceptions*: the syntax is correct, but the flow of the program led to an error.



In [ ]:
str = "this is a syntax error

SyntaxError: unterminated string literal (detected at line 1) (3032787141.py, line 1)

In [3]:
# This is an exception
for a in [3,2,1,0]:
    b = 1/a

ZeroDivisionError: division by zero

In the error message above, python tells us what error occurred ('ZeroDivisionError: division by zero') and where it occurred 'Cell In[3], line 3'.

Consider the following code for quadratic equations:

In [ ]:
import math
def quadraticFormula(a,b,c):
    ''' 
    Returns the solution of the quadratic equation ax^2+bx+c = 0
    '''
    Delta = b**2    - 4 * a * c
    return (-b - math.sqrt(Delta)) / 2. / a, (-b + math.sqrt(Delta)) / 2. / a

print(quadraticFormula(1,-2,1))

(1.0, 1.0)


We know that the program will fail if Delta < 0:

In [5]:
print(quadraticFormula(1,-2,10))

ValueError: math domain error

This is easy to fix: we could test if Delta is negative and return `None` if it is. 

In [13]:
import math
def quadraticFormula(a,b,c):
    ''' 
    Returns the solution of the quadratic equation ax^2+bx+c = 0
    '''
    Delta = b**2    - 4 * a * c
    if Delta >= 0:
        return (-b - math.sqrt(Delta)) / 2. / a, (-b + math.sqrt(Delta)) / 2. / a

print(quadraticFormula(1,-2,10))

None


Another way would be to *try* to compute `math.sqrt(Delta)` and if it fails, handle the *exception*:

In [ ]:
import math
def quadraticFormula(a,b,c):
    ''' 
    Returns the solution of the quadratic equation ax^2+bx+c = 0
    '''        
    Delta = b**2    - 4 * a * c
    try:
        return (-b - math.sqrt(Delta)) / 2. / a, (-b + math.sqrt(Delta)) / 2. / a
    except ValueError as err:
        print("Something went wrong: ", err)
    
    
print(quadraticFormula(1,-2,1))
print(quadraticFormula(1,-2,10))

(1.0, 1.0)
Something went wrong:  math domain error
None


The rationale for doing to is quite weak here, but there may be situations where it is not that easy to predict if an operation will fail or no. Opening a file is one of these:

In [4]:
def readFile(filename):
    f = open(filename)
    return f.read()

for f in ("fileThatDoesNotExist.txt", "example.txt"):
    text = readFile(f)
    print(text)

FileNotFoundError: [Errno 2] No such file or directory: 'fileThatDoesNotExist.txt'

Of course, we could try to check if the file exists (there are ways to do so). We can also *catch* the exception `FileNotFoundError` if it arises and warn the user ahead of time:

In [3]:
def readFile(filename):
    try:
        print(f"Trying to open {filename}")
        f = open(filename)
        return f.read()
    except FileNotFoundError as err:
        print(err)

for f in ("fileThatDoesNotExist.txt", "example.txt"):
    text = readFile(f)
    print(text)

Trying to open fileThatDoesNotExist.txt
[Errno 2] No such file or directory: 'fileThatDoesNotExist.txt'
None
Trying to open example.txt
The quick brown fox jumps over the lazy dog
a man a plan a canal panama


By catching the `FileNotFoundError` exception, we allowed our program to continue and read the second file instead of stopping.

The general syntax for exception catching is the following:
```
try:
   #statements
   :
except (<list1>):
   # What to do if an exception in list1 is caught
   :
except (<list2>): 
    # What to do if an exception in list2 is caught
else:
    # do something if no exceptions are caught
finally:
    # this is always executed
```


In [25]:
import math
for i in (1, 0, -1):
    try:
        print(f"Trying {i}")
        b = 1/i
        c = math.sqrt(i)
    except(ZeroDivisionError):
        print("  caught exception ZeroDivisionError")
        b = 0
    except(ValueError):
        print("  caught exception ValueError")
        c = 0
    else:
        print("  no exception here")
    finally:
        print ("  finally")
    print("done with exception handling")

Trying 1
  no exception here
  finally
done with exception handling
Trying 0
  caught exception ZeroDivisionError
  finally
done with exception handling
Trying -1
  caught exception ValueError
  finally
done with exception handling


Exceptions can also be used, in a function, to report errors that can be handled by the program that called the function.
Going back to the quadratic formula example, we could notify the caller that the equation does not admit roots if the discriminant is negative. 

In [32]:
import math
def quadraticFormula(a,b,c):
    ''' 
    Returns the solution of the quadratic equation ax^2+bx+c = 0
    '''
    Delta = b**2    - 4 * a * c
    if Delta < 0:
        raise ArithmeticError("negative discriminant")
    return (-b - math.sqrt(Delta)) / 2. / a, (-b + math.sqrt(Delta)) / 2. / a

print(quadraticFormula(1,-2,1))
print(quadraticFormula(1,-2,10))

(1.0, 1.0)


ArithmeticError: negative discriminant

`ArithmeticError` is a python built-in exception. You can find a list of such exceptions [here](https://docs.python.org/3/library/exceptions.html#exception-hierarchy). It is also possible to define your own exceptions, but this is beyond the scope of this class.

Here, we'd expect that whatever program calls our quadraticFormula function would handle the exception (and perhaps try to do something that does not require solving this equation.)

In [51]:
def bisection(f, a, b, tol = 1.0e-4, maxit = 100):
    '''
    solves the equation f(x) = 0 using the bisection method with initial interval (a,b)
    '''
    if f(a) * f(b) > 0:
        raise AttributeError("f({a}) and f({b}) do not have opposite signs")
        return 
    elif f(a) == 0:
        return a,a
    elif f(b) == 0:
        return b,b

    iter = 0
    while abs(b-a) > tol:
        iter += 1
        if iter == maxit:
            raise ArithmeticError(f"unable to converge in {iter} iterations")
        m = (a+b)/2
        if f(a) * f(m) < 0:
            b = m
        elif f(a) * f(m) > 0:
            a = m
        else:
            a = m
            b = m
    return a, b

def secant(f, a, b, tol = 1.0e-4, maxit = 100):
    '''
    solves the equation f(x) = 0 using the secant method with initial interval (a,b)
    '''
    iter = 0
    while abs(b-a) > tol and iter < maxit:
        iter += 1
        if f(a) == f(b):
            raise ArithmeticError(f"secant method failed with f({a:2.2f}) = f({b:2.2f}) = {f(a):2.2f}")
        intersect = a - f(a) * (b - a) / (f(b) - f(a))
        a = b
        b = intersect
    return a, b

For instance, the following code tries the bisection method and if it does not wok or does not converge, switches to the secant method (this is of course bogus, since we know exactly how fast the bisection method works, so that we can make sure tat it will converge if the initial interval brackets the solution...)

In [58]:
def f(x):
    return 1-x**2

for a,b in [(-2,2), (0.5,2), (-2,-1.9)]:
    print(f"trying to solve in ({a},{b})")
    try:
        a0,b0 = bisection(f, a, b, 1e-4, 20)
        print(f"   bisection returned ({a0:2.2f},{b0:2.2f})")
    except (ArithmeticError, AttributeError):
        print("   bisection failed")
        try:
            a0,b0 = secant(f, a, b, 1e-4, 5)
        except ArithmeticError:
            print("   secand failed")
        else:
            print(f"   secant returned ({a0:2.2f},{b0:2.2f})")


trying to solve in (-2,2)
   bisection failed
   secand failed
trying to solve in (0.5,2)
   bisection returned (1.00,1.00)
trying to solve in (-2,-1.9)
   bisection failed
   secant returned (-1.00,-1.00)


### The `with` construction

Long version:

One possible issue with catching exceptions is that it may lead to side effects.
See the following example:

```
f = open("somefile.txt", "r")
# do something
: 
:
try:
    # do something else that may lead to an exception
    # :
except:
    # deal with the exception
```
If the exception is caught, then the file "somefile.txt" will be left open and trying to re-open it, or trying to write in it from another program may fail. In some cases, this can be avoided by closing the file in a `finally` statement:
```
f = open("somefile.txt", "r")
# do something
# : 
# :
try:
    # do something else that may lead to an exception
    # :
except:
    # deal with the exception
finally:
    f.close()
```

But this could still leave situations where the `finally` statement is not executed. This could happen, for instance, if the program calls a functions which itself handles exceptions of craches in the "try" statements.

The `with ... as` construction is designed to manage this type of situation.  By far, its most common use is when dealing with files or external commands (which we will not discuss in this class), and the syntax is:

```
with open("somefile.txt") as f:
    data = f.read()
    # do something with data
```
In this case, you do not need to explicitly close the file with `f.close()`. Python does it for you.

**TLDR; version:**

Do not write
```
f = open("somefile.txt)
# do something with f
f.close()
```
but instead
``` 
with open("somefile.txt) as f:
    # do something with f
```